# Machine learning-based automated classification of headache disorders using patient-reported questionnaires

### Configuration

In [1]:
# A csv file containing feature appearance information.
# [model_index] should be within the range from 1 to [num_models].
# [fold_index] should be within the range from 1 to [num_folds].
feature_appearance_csv_filename = "data/model_{model_index}_fold_{fold_index}.csv"

# Train and test csv data for each fold.
# [fold_index] should be within the range from 1 to [num_folds].
train_csv_filename = "data/train.csv"
test_csv_filename = "data/test.csv"

# A parameter to get stable features for each fold.
# Stable feature means the one appeared at least [appearance_threshold] times out of 100 LASSO trials.
appearance_threshold = 95

# Learning rate for XGBClassifier (default: 0.1)
learning_rate = 0.06

### Constants

In [2]:
# Stacked XGBoost classifiers
# First model: Migraine vs Non-migraine
# Second model: Tension-type headache vs Non-TTH
# Third model: Epicranial headache vs Thunderclap headache
num_models = 3

# Parameter K for K-fold cross validation.
num_folds = 10

# Subsequent class index
class_dict = {
    "migraine": 1,
    "tension-type headache": 2,
    "epicranial headache": 4,
    "thunderclap headache": 5
}

# Dominant classes for each model
class_indices = (
    class_dict["migraine"],
    class_dict["tension-type headache"],
    class_dict["epicranial headache"]
)

### The code

In [3]:
from typing import List
from collections import Counter
import random

import numpy as np
import pandas as pd
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

from utils import *

### Read feature appearance information for each K-fold.

In [4]:
# ---------------------------------------------------------------------------------
# 1. Read feature appearance information
# ---------------------------------------------------------------------------------
features_demo = []
for array_index in range(num_models):
    model_index = array_index + 1
    features_demo.append([])
    for fold_index in range(1, num_folds + 1):
        features_demo[array_index] += read_feature_appearance(
            filename=feature_appearance_csv_filename.format(model_index=model_index, fold_index=fold_index),
            threshold=appearance_threshold
        )
    features_demo[array_index] = Counter(features_demo[array_index]).most_common()

### Select features, train models, and evaluate the models for each fold.

In [5]:
# For each fold, [test_accuracy, test_min_sensitivity, test_min_specificity] will be stored in [evaluation_matrix].
evaluation_matrix = np.zeros((num_folds, 3))
for selection_threshold in range(1, num_folds + 1):
    # -----------------------------------------------------------------------------
    # 2. Filter features
    # -----------------------------------------------------------------------------
    selected_features = []
    should_skip = False
    for features in features_demo:
        sub_features_list = []
        for feature_name, num_selected_times in features:
            if num_selected_times >= selection_threshold:
                sub_features_list.append(feature_name)
        if len(sub_features_list) == 0:
            should_skip = True
            break
        selected_features.append(sub_features_list)
    if should_skip:
        print(f"Skip for stage {selection_threshold} as one of layers has no feature available,")
        evaluation_matrix[selection_threshold - 1] = [-1, -1, -1]
        continue
    # -----------------------------------------------------------------------------
    # 3. Train a model
    # -----------------------------------------------------------------------------
    models = []
    for array_index in range(num_models):
        # When no feature is selected
        model_index = array_index + 1
        X_train, y_train, _ = read_demographics(train_csv_filename)
        # Drop unnecessary subjects for each model.
        if model_index == 2:  # TTH classifier
            X_train, y_train = drop_subjects_by_classes(X_train, y_train, [class_dict["migraine"]])
        elif model_index == 3:  # Specific headache syndromes classifier
            X_train, y_train = drop_subjects_by_classes(X_train, y_train,
                                                        [class_dict["migraine"], class_dict["tension-type headache"]])
        # Adjust Y
        y_train = (y_train == class_indices[array_index]).astype(np.int32)
        # Adjust X
        X_train = X_train.loc[:, selected_features[array_index]].sort_index(axis=1)
        # Train a model
        model = XGBClassifier(objective='binary:logistic', learning_rate=learning_rate)
        model.fit(X_train, y_train, eval_metric="auc")
        models.append(model)
    # -----------------------------------------------------------------------------
    # 4. Evaluate the model using independent dataset
    # -----------------------------------------------------------------------------
    X_test, y_test, _ = read_demographics(test_csv_filename)
    migraine_index = 0
    tth_index = 1
    specific_syndrome_index = 2
    # (1) Migraine classifier
    X_test_migraine = X_test.loc[:, selected_features[migraine_index]].sort_index(axis=1)
    y_pred_migraine = models[migraine_index].predict(X_test_migraine)
    # (2) TTH classifier
    tth_indices = np.where(y_pred_migraine != 1)[0]
    X_test_tth = X_test.iloc[tth_indices, :].loc[:, selected_features[tth_index]].sort_index(axis=1)
    y_pred_tth = models[tth_index].predict(X_test_tth)
    # (3) Specific headache syndromes classifier
    specific_syndrome_indices = tth_indices[np.where(y_pred_tth != 1)[0]]
    X_test_specific_syndrome = X_test.iloc[specific_syndrome_indices, :].loc[:,
                               selected_features[specific_syndrome_index]].sort_index(axis=1)
    y_pred_specific_syndrome = models[specific_syndrome_index].predict(X_test_specific_syndrome)
    # Adjust prediction class
    y_pred = y_pred_migraine.copy()
    y_pred[y_pred_migraine == 1] = class_indices[migraine_index]
    y_pred[tth_indices[y_pred_tth == 1]] = class_indices[tth_index]
    if class_indices[specific_syndrome_index] == class_dict["epicranial headache"]:
        y_pred[specific_syndrome_indices[y_pred_specific_syndrome == 0]] = class_dict["thunderclap headache"]
        y_pred[specific_syndrome_indices[y_pred_specific_syndrome == 1]] = class_dict["epicranial headache"]
    else:
        y_pred[specific_syndrome_indices[y_pred_specific_syndrome == 0]] = class_dict["epicranial headache"]
        y_pred[specific_syndrome_indices[y_pred_specific_syndrome == 1]] = class_dict["thunderclap headache"]
    # Evaluate scores
    test_accuracy = np.mean(y_pred == y_test)
    test_conf_mat = confusion_matrix(y_test, y_pred)
    # Get minimum sensitivity and specificity.
    test_min_sensitivity = 1
    test_min_specificity = 1
    for i in range(test_conf_mat.shape[0]):
        sensitivity = test_conf_mat[i, i] / np.sum(test_conf_mat[:, i])
        test_min_sensitivity = min(sensitivity, test_min_sensitivity)
        specificity = test_conf_mat[i, i] / np.sum(test_conf_mat[i, :])
        test_min_specificity = min(specificity, test_min_specificity)
    # -----------------------------------------------------------------------------
    # 5. Save evaluated scores in [evaluation_matrix].
    # -----------------------------------------------------------------------------
    evaluation_matrix[selection_threshold - 1] = [test_accuracy, test_min_sensitivity, test_min_specificity]

### Test Result
Negative value means the corresponding fold has been skipped.

In [6]:
print(pd.DataFrame(evaluation_matrix,
                   index=[f"Fold {i}" for i in range(1, num_folds + 1)],
                   columns=["Accuracy", "Sensitivity", "Specificity"]))

         Accuracy  Sensitivity  Specificity
Fold 1   0.838828     0.500000     0.377049
Fold 2   0.836386     0.446809     0.344262
Fold 3   0.837607     0.500000     0.393443
Fold 4   0.840049     0.488889     0.360656
Fold 5   0.835165     0.468085     0.360656
Fold 6   0.836386     0.483871     0.377049
Fold 7   0.837607     0.488889     0.360656
Fold 8   0.836386     0.440000     0.360656
Fold 9   0.825397     0.418605     0.295082
Fold 10  0.822955     0.372093     0.262295
